<a href="https://colab.research.google.com/github/bytehub-ai/code-examples/blob/main/bytehub_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q bytehub

     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 122kB 13.4MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import bytehub as bh
import plotly.graph_objects as go

In [15]:
fs = bh.FeatureStore(
    endpoint='https://api.dev.bytehub.ai/v1',
    client_id='rlnnlteca0oq01bk081kivg4s'
)

Please go to https://bytehub-ai-dev.auth.eu-west-2.amazoncognito.com/login?response_type=code&client_id=rlnnlteca0oq01bk081kivg4s&redirect_uri=https%3A%2F%2Fwww.bytehub.ai%2Fauthenticated&state=uS0uItkwTKhy5ONBg9wYH43MReHNJm and login. Copy the response code and paste below.
Response: ··········


## List features in the feature store

In [16]:
# Search for the BMRS features
fs.list_features(regex=r'bmrs\..')

['bmrs.data.bod',
 'bmrs.feature.lowest-offer',
 'bmrs.feature.highest-bid',
 'bmrs.data.indoitsdo',
 'bmrs.feature.indo',
 'bmrs.feature.itsdo',
 'bmrs.data.fuelhh',
 'bmrs.feature.wind-generation',
 'bmrs.data.dersysdata',
 'bmrs.feature.system-price',
 'bmrs.feature.niv',
 'bmrs.data.rolsysdem',
 'bmrs.feature.rolling-demand',
 'bmrs.feature.demand-30min-history']

## Get some features and plot a chart

In [17]:
df = fs.get_timeseries(['bmrs.feature.system-price', 'bmrs.feature.niv'], from_date='2020-09-01', to_date=pd.Timestamp.utcnow(), freq='30min')
df.head()

,time,entity,bmrs.feature.system-price,bmrs.feature.niv
0,2020-09-01 00:00:00+00:00,None,46.4,249.9333
1,2020-09-01 00:30:00+00:00,None,46.3,335.4755
2,2020-09-01 01:00:00+00:00,None,46.3,437.5139
3,2020-09-01 01:30:00+00:00,None,46.0,239.6694
4,2020-09-01 02:00:00+00:00,None,46.0,316.8444


In [6]:
traces = [
          go.Scatter(x=df.time, y=df['bmrs.feature.system-price'], name='System Price'),
          go.Scatter(x=df.time, y=df['bmrs.feature.niv'], name='NIV', yaxis='y2'),
]
layout = {
    'title': 'System Prices and NIV',
    'template': 'seaborn',
    'yaxis': {'title': 'System price'},
    'yaxis2': {'title': 'Net Imbalance Volume', 'overlaying': 'y', 'side': 'right'},
}
fig = go.Figure(data=traces, layout=layout)
fig

In [7]:
# Upsample
fs.get_timeseries(['bmrs.feature.system-price', 'bmrs.feature.niv'], from_date='2020-09-01', to_date=pd.Timestamp.utcnow(), freq='10min').head()

,time,entity,bmrs.feature.system-price,bmrs.feature.niv
0,2020-09-01 00:00:00+00:00,None,46.4,249.9333
1,2020-09-01 00:10:00+00:00,None,46.4,249.9333
2,2020-09-01 00:20:00+00:00,None,46.4,249.9333
3,2020-09-01 00:30:00+00:00,None,46.3,335.4755
4,2020-09-01 00:40:00+00:00,None,46.3,335.4755


In [8]:
# Downsample
fs.get_timeseries(['bmrs.feature.system-price', 'bmrs.feature.niv'], from_date='2020-09-01', to_date=pd.Timestamp.utcnow(), freq='1d').head()

,time,entity,bmrs.feature.system-price,bmrs.feature.niv
0,2020-09-01 00:00:00+00:00,None,46.4,249.9333
1,2020-09-02 00:00:00+00:00,None,15.0,-4.6359
2,2020-09-03 00:00:00+00:00,None,51.5,231.7499
3,2020-09-04 00:00:00+00:00,None,2.0,-401.1833
4,2020-09-05 00:00:00+00:00,None,2.0,-128.9746


## More complex features

In [9]:
df = fs.get_timeseries('bmrs.data.dersysdata', from_date='2020-10-01', to_date='2020-10-03', freq='30min')
df.tail()

,time,entity,bmrs.data.dersysdata
92,2020-10-02 22:00:00+00:00,None,"[{'time': 1601676000000, 'activeFlag': 'Y', 'r..."
93,2020-10-02 22:30:00+00:00,None,"[{'time': 1601677800000, 'activeFlag': 'Y', 'r..."
94,2020-10-02 23:00:00+00:00,None,"[{'time': 1601679600000, 'activeFlag': 'Y', 'r..."
95,2020-10-02 23:30:00+00:00,None,"[{'time': 1601681400000, 'activeFlag': 'Y', 'r..."
96,2020-10-03 00:00:00+00:00,None,"[{'time': 1601683200000, 'activeFlag': 'Y', 'r..."


In [10]:
df.iloc[-1, -1]

[{'activeFlag': 'Y',
  'bSADDefault': 'F',
  'buyPriceAdjustment': 0,
  'indicativeNetImbalanceVolume': 16.363,
  'priceDerivationCode': 'P',
  'recordType': 'SSB',
  'reserveScarcityPrice': 'NULL',
  'sellPriceAdjustment': 0,
  'settlementDate': 1601683200000,
  'settlementPeriod': 3,
  'systemBuyPrice': 50.0,
  'systemSellPrice': 50.0,
  'time': 1601683200000,
  'totalSystemAcceptedBidVolume': -368.035,
  'totalSystemAcceptedOfferVolume': 908.898,
  'totalSystemAdjustmentBuyVolume': 437.0,
  'totalSystemAdjustmentSellVolume': -961.5,
  'totalSystemTaggedAcceptedBidVolume': -368.035,
  'totalSystemTaggedAcceptedOfferVolume': 907.898,
  'totalSystemTaggedAdjustmentBuyVolume': 437.0,
  'totalSystemTaggedAdjustmentSellVolume': -961.5}]

In [18]:
fs.get_last('bmrs.data.dersysdata')

,time,entity,value
0,2020-10-05 22:30:00+00:00,None,"[{'time': 1601937000000, 'activeFlag': 'Y', 'r..."


In [12]:
fs.get_last('bmrs.data.rolsysdem').value[0]

[{'activeFlag': 'Y',
  'fuelTypeGeneration': 22399,
  'publishingPeriodCommencingTime': '2020-10-06 00:00:00',
  'recordType': 'VD',
  'settDate': '2020-10-06',
  'time': 1601942400000}]

In [13]:
fs.get_last('bmrs.feature.demand-30min-history')

,time,entity,value
0,2020-10-06 00:00:00+00:00,None,"[22671, 22610, 22566, 22520, 22477, 22399]"


## Create a new feature

In [ ]:
fs.create_feature('test.my-new-feature', source='demo', animal='dog')

In [ ]:
dts = pd.date_range('2020-10-01', '2020-10-04', freq='1h')
df = pd.DataFrame({'time': dts, 'value': np.random.randint(0, 100, len(dts))})
fs.save_timeseries('test.my-new-feature', df)

In [ ]:
fs.get_timeseries('test.my-new-feature', from_date='2020-09-25', to_date='2020-10-04', freq='1d')

,time,entity,test.my-new-feature
0,2020-09-25 00:00:00+00:00,None,NaN
1,2020-09-26 00:00:00+00:00,None,NaN
2,2020-09-27 00:00:00+00:00,None,NaN
3,2020-09-28 00:00:00+00:00,None,NaN
4,2020-09-29 00:00:00+00:00,None,NaN
5,2020-09-30 00:00:00+00:00,None,NaN
6,2020-10-01 00:00:00+00:00,None,92.0
7,2020-10-02 00:00:00+00:00,None,15.0
8,2020-10-03 00:00:00+00:00,None,11.0
9,2020-10-04 00:00:00+00:00,None,90.0


In [ ]:
fs.delete_feature('test.my-new-feature')